In [1]:
import pandas as pd 
import eurostat
df = eurostat.get_data_df('sbs_sc_sca_r2', flags=False)
df

,nace_r2,indic_sb,size_emp,geo\time,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,B,V11110,0-9,AT,NaN,183.0,192.0,215.0,226.0,230.0,227.0,238.0,238.0,242.0,245.0,244.0,237.0,NaN,NaN,NaN
1,B,V11110,0-9,BA,NaN,101.0,102.0,121.0,129.0,132.0,121.0,118.0,121.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN
2,B,V11110,0-9,BE,NaN,140.0,118.0,173.0,144.0,167.0,149.0,175.0,160.0,227.0,249.0,248.0,NaN,NaN,NaN,NaN
3,B,V11110,0-9,BG,NaN,190.0,211.0,215.0,224.0,240.0,228.0,243.0,252.0,245.0,241.0,230.0,199.0,NaN,NaN,NaN
4,B,V11110,0-9,CH,NaN,82.0,84.0,81.0,85.0,84.0,86.0,89.0,75.0,75.0,79.0,85.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475052,S95,V92100,TOTAL,RS,NaN,4.4,4.3,4.2,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475053,S95,V92100,TOTAL,SE,NaN,1.1,1.1,1.2,1.2,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475054,S95,V92100,TOTAL,SI,NaN,1.5,1.5,1.5,1.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475055,S95,V92100,TOTAL,SK,NaN,1.5,1.4,1.5,1.4,1.4,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#eliminate columns for years previous to 2011
years =[]
for y in range(2005,2011): 
    years.append(y)
df=df.drop(years, axis=1)#drop columns with years before 2011
#rename geo time column to be easier to handle because of the name
df.rename(columns={"geo\\time": "geo"}, inplace=True)

In [3]:
#FUNCTION CALLABLE TO SELECT FOR EXAMPLE YEARS AND VARIABLE
import PySimpleGUI as sg
def selection(option):
    #sg.theme('DarkAmber')   # Add a touch of color
    #select the variable we want to calculate
    options = option

    # All the stuff inside your window.
    layout = [ 
            [sg.Text('Select one->'), sg.Listbox(options,select_mode=sg.LISTBOX_SELECT_MODE_SINGLE,size=(20,len(options)))],
            [sg.Button('Ok'), sg.Button('Cancel')]
        ]

    # Create the Window
    window = sg.Window('Make your choice', layout)

    # Event Loop to process "events" and get the "values" of the input
    while True:
        event, values = window.read()
        print( f"event={event}" )
        if event is None or event == 'Ok' or event == 'Cancel': # if user closes window or clicks cancel
            break
        
    # close  the window        
    window.close()

    if event == "Cancel":
        print( "You cancelled" )
    else:
        print('You entered ', values[0])
        sg.popup( f"You selected {values[0]}" )        
    return values

In [4]:
#selection of the variable (Calling the function)
dic_indic_sb = eurostat.get_dic('indic_sb') #dictionary for all the indicators in sbs
for indic in df.indic_sb.unique():
    if indic in dic_indic_sb:
        print (indic + " -"+dic_indic_sb[indic])

var=selection(df.indic_sb.unique())[0]
#determine the variable of interest based on the selection
for indic in var:
    if indic in dic_indic_sb:
        variable_of_interest=dic_indic_sb[indic]

print(variable_of_interest)
print(" ")


#create a list of years for selection
yearsloop =[]
for y in range(2011,2021): 
    yearsloop.append(y)
#select de year frame (Calling the function) with the starting year and the end year
year_start=selection(yearsloop)[0] #with [] we convert variable to a list to apply "isin"
year_end=selection(yearsloop)[0]
print(year_start)
print(year_end)

V11110 -Enterprises - number
V12110 -Turnover or gross premiums written - million euro
V12120 -Production value - million euro
V12150 -Value added at factor cost - million euro
V12170 -Gross operating surplus - million euro
V13110 -Total purchases of goods and services - million euro
V13310 -Personnel costs - million euro
V13320 -Wages and Salaries - million euro
V13330 -Social security costs - million euro
V16110 -Persons employed - number
V16120 -Unpaid persons employed - number
V16130 -Employees - number
V16150 -Hours worked by employees - number
V91100 -Turnover per person employed - thousand euro
V91110 -Apparent labour productivity (Gross value added per person employed) - thousand euro
V91120 -Wage adjusted labour productivity (Apparent labour productivity by average personnel costs) - percentage
V91130 -Gross value added per employee - thousand euro
V91150 -Gross value added per hour worked by employees
V91170 -Share of personnel costs in production - percentage
V91210 -Average

In [5]:
#select nace
nace_sel=['C31', 'C32', 'N80','N81','N82']

#countries to eliminate
countries_not_selected= ['UK', 'TR', 'AL','IS','LI','NO','CH','UK','ME','MK','AL','RS','TR','BA','EU28','EU15','EA','EA19','EA12', 'EU27_2007']
# columns of countries to be eliminated after sum
cols = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI','FR', 'HR', 'HU', 'IE', 'IT', 'LT','LU', 'LV', 'MT','NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']

#size total values (no distribution by size)
size_sel=['TOTAL']#we select the totals

In [6]:
import numpy as np
results_tot = pd.DataFrame() #create an empty dataframe for total results
#filter by size, nace, variable and countries
df_selection=df[df.size_emp.isin(size_sel)&df.nace_r2.isin(nace_sel)&df.indic_sb.isin(var)&~df.geo.isin(countries_not_selected)]
df_selection

,nace_r2,indic_sb,size_emp,geo,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
312528,C31,V12150,TOTAL,AT,NaN,1399.7,1383.4,1383.2,1372.7,1282.5,1261.5,1210.8,1248.7,1229.1
312530,C31,V12150,TOTAL,BE,NaN,680.2,646.1,588.7,644.1,653.9,666.4,641.2,634.6,666.6
312531,C31,V12150,TOTAL,BG,NaN,216.2,210.7,170.0,166.9,148.0,131.3,119.2,115.2,99.7
312533,C31,V12150,TOTAL,CY,NaN,26.5,23.7,21.9,19.0,17.6,18.4,18.7,22.3,28.2
312534,C31,V12150,TOTAL,CZ,NaN,519.8,490.9,481.1,454.3,413.5,369.1,368.7,383.6,374.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472378,N82,V12150,TOTAL,PT,NaN,2417.0,2437.3,2293.8,2019.6,1815.9,1681.7,1514.5,1498.5,1551.6
472379,N82,V12150,TOTAL,RO,NaN,1360.2,1158.3,1035.4,840.3,697.6,688.7,565.4,428.0,424.9
472381,N82,V12150,TOTAL,SE,NaN,2074.2,2013.2,1249.0,1305.1,1447.9,1520.2,1573.6,1346.4,1244.5
472382,N82,V12150,TOTAL,SI,NaN,105.2,102.9,86.8,71.1,55.2,46.6,37.9,37.0,43.1


In [7]:
#iteration to retrieve the data for each year
for i in range (year_start[0],year_end[0]+1):
    results = pd.DataFrame() #create an empty dataframe for partial results
    df_target = pd.pivot_table(df_selection, values=i, index=['nace_r2'],columns=["geo"], aggfunc=np.sum, fill_value=0)#rearrange using nace
    df_target['sum'] = df_target[cols].sum(axis=1)#sums the columns of the countries to "sum"
    df_target=df_target.drop(cols, axis=1)#eliminate countries columns
    #now we can locate columns with 0 and make the substitution with the column "sum"
    df_target.loc[df_target['EU27_2020']==0,'EU27_2020'] = df_target['sum']#adds summ if column is empty
    moddf_target=df_target.drop('sum', axis=1)#drop sum column
    column_name=str(variable_of_interest)+str(i)#determine de column name using variable of interest and year
    results[column_name]= moddf_target['EU27_2020']
    results_tot = pd.concat([results_tot,results], axis=1)#adds column in each iteration (for each year)
results_tot

,Value added at factor cost - million euro2011,Value added at factor cost - million euro2012,Value added at factor cost - million euro2013,Value added at factor cost - million euro2014,Value added at factor cost - million euro2015,Value added at factor cost - million euro2016,Value added at factor cost - million euro2017,Value added at factor cost - million euro2018,Value added at factor cost - million euro2019
nace_r2,,,,,,,,,
C31,25942.9,25351.6,25126.6,25992.3,27118.8,29000.0,29953.3,31000.0,32000.0
C32,39156.6,37505.3,37603.5,39289.1,40551.1,41874.3,47133.7,50000.0,50000.0
N80,24979.4,25000.0,25126.8,25293.1,27000.0,29108.0,29828.8,31000.0,32000.0
N81,69700.0,70543.8,74000.0,77019.1,79840.2,84973.4,90000.0,101568.8,107844.9
N82,56000.0,58000.0,59419.4,63732.8,64085.4,67738.3,71860.2,80000.0,80000.0


In [8]:
ratio_results = results_tot.copy() #create a dataframe for the ratios (copying the result_tot)
years= year_end[0]-year_start[0]
for i in range (0,years+1) :
    ratio_results.iloc[0][i]=results_tot.iloc[0][i]/(results_tot.iloc[0][i]+results_tot.iloc[1][i])
    ratio_results.iloc[1][i]=results_tot.iloc[1][i]/(results_tot.iloc[0][i]+results_tot.iloc[1][i])
    ratio_results.iloc[2][i]=results_tot.iloc[2][i]/(results_tot.iloc[2][i]+results_tot.iloc[3][i]+results_tot.iloc[4][i])
    ratio_results.iloc[3][i]=results_tot.iloc[3][i]/(results_tot.iloc[2][i]+results_tot.iloc[3][i]+results_tot.iloc[4][i])
    ratio_results.iloc[4][i]=results_tot.iloc[4][i]/(results_tot.iloc[2][i]+results_tot.iloc[3][i]+results_tot.iloc[4][i])
    #_tot.iloc[0][1]#=ratio_tot.iloc[0][i](ratio_tot.iloc[0][i]+ratio_tot.iloc[1][i])
ratio_results    
    #=ratio_tot.iloc["C31"][i]/(ratio_tot.iloc["C31", i]+ratio_tot.iloc["C32", i])


,Value added at factor cost - million euro2011,Value added at factor cost - million euro2012,Value added at factor cost - million euro2013,Value added at factor cost - million euro2014,Value added at factor cost - million euro2015,Value added at factor cost - million euro2016,Value added at factor cost - million euro2017,Value added at factor cost - million euro2018,Value added at factor cost - million euro2019
nace_r2,,,,,,,,,
C31,0.398512,0.403322,0.400551,0.398158,0.400751,0.409175,0.388565,0.382716,0.390244
C32,0.601488,0.596678,0.599449,0.601842,0.599249,0.590825,0.611435,0.617284,0.609756
N80,0.165778,0.162820,0.158483,0.152327,0.157963,0.160093,0.155610,0.145835,0.145557
N81,0.462572,0.459438,0.466741,0.463845,0.467105,0.467350,0.469511,0.477816,0.490550
N82,0.371650,0.377742,0.374777,0.383828,0.374932,0.372558,0.374879,0.376349,0.363893


In [9]:
#save to file to calculate ration
fileratio="ratioCN-"+ variable_of_interest
filename = "%s.xlsx" % fileratio
ratio_results.to_excel(filename)